In [13]:
from dotenv import load_dotenv

load_dotenv()

True

In [14]:
from langchain.chat_models import init_chat_model


model = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

In [15]:
from langchain_core.messages import HumanMessage

ai_message = model.invoke([HumanMessage(content="Hi! I'm Dan.")])

print(ai_message.content)

Hi Dan! Nice to meet you. How can I help you today?


In [21]:
# ai_message = model.invoke([HumanMessage(content="Training cutoff date for Gemini 2.5 Flash?")])
ai_message = model.invoke([HumanMessage(content="What's my name?")])

print(ai_message.content)

I don't know your name. As an AI, I don't have access to your personal information or any details about who you are unless you choose to tell me during our conversation.
